In [1]:
%run ../../notebooks/__init__.ipynb

In [2]:
def read_pssm(struct_id):
    #zip_ = '../../results/foldx/af2.repairpdb.pssm/Q9/Y5/Z9/Q9Y5Z9.zip'
    zip_ = pfile(struct_id=struct_id, step='af2.trim_bf.repairpdb.pssm', suffix='.zip', base='../../results/human/')
    txt_ = 'individual_list_0_PSSM.txt'
    avg_ = f'Average_{struct_id}.fxout'
    with zipfile.ZipFile(zip_) as fh_zip_:
        with fh_zip_.open(txt_) as fh_txt_:
            df_pos_ = pd.read_csv(fh_txt_, names=['pssm_pos'])
        with fh_zip_.open(avg_) as fh_avg_:
            df_avg_ = pd.read_csv(fh_avg_, sep='\t')

    def parse_pssm_pos(r):
        aa_pos = int(r.pssm_pos[2:-2])
        aa_ref = r.pssm_pos[0]
        aa_alt = r.pssm_pos[-2]
        chain = r.pssm_pos[1]
        return aa_ref, chain, aa_pos, aa_alt

    df_ = pd.concat([df_pos_, df_avg_], axis=1)
    df_[['aa_ref', 'chain', 'aa_pos', 'aa_alt']] = df_.apply(parse_pssm_pos, axis=1, result_type='expand')
    def apply_(r):
        return f'{struct_id}/{r.aa_ref}{r.aa_pos}{r.aa_alt}'
    df_['variant_id'] = df_.apply(apply_, axis=1)

    cols_ = df_.columns[-5:].tolist() + df_.columns[:-5].tolist()
    df_ = df_[cols_]

    return df_

df_ = pd.concat(map(read_pssm, ['P00533', 'P04626', 'P04637', 'Q15648' ]), axis=0)#.sample(n=1000)
phead(df_)

54,680 records


,0,1,2
aa_ref,K,K,K
chain,A,A,A
aa_pos,29,29,29
aa_alt,A,C,D
variant_id,P00533/K29A,P00533/K29C,P00533/K29D
pssm_pos,KA29A;,KA29C;,KA29D;
Pdb,P00533_1,P00533_2,P00533_3
SD,0,0,0
total energy,0.259316,0.619045,0.402834
Backbone Hbond,1.03902,0.961476,0.366827


In [3]:
frame_clinvar = merge_missense(df_, 'variant_id')
frame_clinvar

merge_missense: 54,680 raw records
merge_missense: 54,680 records matched to predictions
merge_missense: 18,633 annotated as destabilizing
merge_missense: 6,100 annotated with interfaces at pDockQ > 0.23
merge_missense: 27,780 annotated with pockets at pocket_score > 800


,aa_ref,chain,aa_pos,aa_alt,variant_id,pssm_pos,Pdb,SD,total energy,Backbone Hbond,...,Entropy Complex,am_pathogenicity,am_class,am_label,pred_ddg,pred_ddg_label,interface_pdockq,interface_label,pocket_label,mechanistic_label
0,K,A,29,A,P00533/K29A,KA29A;,P00533_1,0,0.259316,1.039020,...,0,0.2953,benign,False,0.754909,False,NaN,False,True,Pockets
1,K,A,29,C,P00533/K29C,KA29C;,P00533_2,0,0.619045,0.961476,...,0,0.8289,pathogenic,True,1.150450,False,NaN,False,True,Pockets
2,K,A,29,D,P00533/K29D,KA29D;,P00533_3,0,0.402834,0.366827,...,0,0.6167,pathogenic,True,1.255870,False,NaN,False,True,Pockets
3,K,A,29,E,P00533/K29E,KA29E;,P00533_4,0,0.363608,0.138926,...,0,0.2175,benign,False,0.799203,False,NaN,False,True,Pockets
4,K,A,29,F,P00533/K29F,KA29F;,P00533_5,0,0.550734,0.845414,...,0,0.7423,pathogenic,True,0.498027,False,NaN,False,True,Pockets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54675,N,A,721,S,Q15648/N721S,NA721S;,Q15648_11136,0,0.264377,0.396696,...,0,0.2149,benign,False,0.120214,False,NaN,False,False,Unassigned
54676,N,A,721,T,Q15648/N721T,NA721T;,Q15648_11137,0,0.195638,0.396696,...,0,0.3061,benign,False,0.092178,False,NaN,False,False,Unassigned
54677,N,A,721,V,Q15648/N721V,NA721V;,Q15648_11138,0,0.338198,0.396696,...,0,0.5061,ambiguous,False,0.257869,False,NaN,False,False,Unassigned
54678,N,A,721,W,Q15648/N721W,NA721W;,Q15648_11139,0,-0.167019,0.396696,...,0,0.9664,pathogenic,True,-0.243830,False,NaN,False,False,Unassigned


In [4]:
frame_clinvar[['total energy', 'pred_ddg']].corr()

,total energy,pred_ddg
total energy,1.000000,0.954368
pred_ddg,0.954368,1.000000
